# <center>  MODELO  de RECOMENDACION </br>
#  <center> de los juegos</center>

____________
__________

Es hora de entrenar nuestro <b><u><i>Modelo de Machine Learning</i></u></b> para armar un sistema de recomendación. Se lleva a cabo la experimentación para desarrollar dos modelos de recomendación capaces de generar listas de '5 juegos'. Estos modelos permiten realizar recomendaciones tanto ingresando el nombre de un juego como el ID de un usuario en Steam.

El primer enfoque utiliza un modelo de <b><u><i>relación ítem-ítem</i></u></b>, donde se evalúa la similitud entre juegos para recomendar aquellos que sean más similares al juego de entrada. Por otro lado, el segundo modelo implementa un <b><u><i>filtro user-item</i></u></b>, identificando usuarios similares y recomendando juegos basándose en las preferencias de esos usuarios afines.

Ambos modelos adoptan algoritmos basados en la memoria, abordando el problema del filtrado colaborativo al utilizar toda la base de datos. Este enfoque busca encontrar usuarios con preferencias similares al usuario activo para predecir sus valoraciones, ofreciendo así recomendaciones personalizadas y relevantes.

________
________

### <center>Importar Librerias</center>

In [1]:
import pandas as pd
import funciones

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd
import os

import warnings
warnings.filterwarnings("ignore")

_____________

### <center>Extracción de datos</center>

 Se procede a la lectura de los datos previamente preparados durante el Análisis Exploratorio de Datos (EDA). Estos datos se transforman en un DataFrame, una estructura tabular que facilita su manipulación y uso por parte del modelo de recomendación.

In [2]:
# Obtén la ruta completa al archivo CSV desde la ubicación del notebook
csv_path = os.path.join('..', 'CSV', 'games_limpio.csv')

# Lee el archivo CSV
df_games = pd.read_csv(csv_path, encoding='utf-8')

___________


## Se traen ciertas columnas de df_games

In [3]:
# Se seleccionan las columnas 'item_id', 'title' y 'genres' del DataFrame df_games
df_games_d= df_games[['item_id', 'title', 'genres']]

# Se visualiza reducidamente
df_games_d.head(2)

,item_id,title,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual


In [4]:
# Se agrupar por título y concatena las entradas de la columna 'genres'
df_agrupado = df_games_d.groupby('title')['genres'].apply(lambda x: ', '.join(x)).reset_index()

# Se visualiza la tabla
df_agrupado

,title,genres
0,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie"
1,"""Barely Attuned Magic Thingy"" Staff","Action, Free to Play, Indie, Massively Multipl..."
2,"""Glow Ball"" - The billiard puzzle game","Casual, Indie, Sports, Strategy"
3,"""Just Another Day"" - Seduce Me Otome CD","Casual, Simulation"
4,"""Lethargic Sentience"" Wand","Action, Free to Play, Indie, Massively Multipl..."
...,...,...
28822,（尘沙惑设定集）Lost in Secular Love - Concept Design ...,"Adventure, Casual, Indie, Simulation"
28823,４人打ちアクション麻雀 / ACTION MAHJONG,"Action, Casual, Indie, Early Access"
28824,＜/reality＞,"Adventure, Indie"
28825,＜/reality＞ Original Soundtrack,"Adventure, Indie"


____________


### <center> Verificación de tipos de datos </center>

In [5]:
# Se verifican los tipos de datos de las columnas y los nulos
funciones.verificar_tipo_datos(df_agrupado)

,Columna,Tipo,NO_nulos_%,Nulos_%,Nulos
0,title,[<class 'str'>],100.0,0.0,0
1,genres,[<class 'str'>],100.0,0.0,0


_________

### <center> Verificación de duplicados </center>

In [6]:
# Se verificar duplicados
hay_duplicados = df_agrupado.duplicated().any()

# Imprimir resultado
if hay_duplicados:
    print("Hay filas duplicadas en el DataFrame.")
else:
    print("No hay filas duplicadas en el DataFrame.")

No hay filas duplicadas en el DataFrame.


_________

### <center> Generación del modelo </center>

In [7]:
# Se crea un objeto CountVectorizer con un límite de 5000 características y se excluyen las palabras comunes en inglés
cv = CountVectorizer(max_features = 5000, stop_words="english")

In [8]:
# Se aplica el CountVectorizer al conjunto de datos en la columna 'genres' y se transforma en una matriz
# La forma resultante se imprime para mostrar el número de filas (documentos) y columnas (características)
cv.fit_transform(df_agrupado["genres"]).toarray().shape

(28827, 32)

In [9]:
# Se generan los vectores a comparar 
cv.fit_transform(df_agrupado["genres"]).toarray().shape
vectores = cv.fit_transform(df_agrupado["genres"]).toarray()

In [10]:
# Se observan la forma de los vectores
vectores

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [11]:
# Se aplica la similitud del coseno a nuestros vectores
similitud = cosine_similarity(vectores)

In [12]:
# Se obtiene el array de similitud
similitud[0]

array([1.        , 0.37796447, 0.5       , ..., 0.70710678, 0.70710678,
       0.70710678])

In [13]:
# Se ordena la similitud entre más similar a menos similar tomando 5 valores
sorted(list(enumerate(similitud[0])), reverse=True, key=lambda x:x[1])[1:6]

[(152, 1.0), (306, 1.0), (488, 1.0), (628, 1.0), (629, 1.0)]

In [14]:
# Se genera una funcion que te da la recomendación por título
def recomendacion(game):
    indice_juegos = df_agrupado[df_agrupado["title"]==game].index[0]
    distances = similitud[indice_juegos]
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_titles = [df_agrupado.iloc[i[0]]['title'] for i in lista_juegos]
    return recommended_titles

_______________________

### <center> Comprobación del funcionamiento del modelo</center>

In [ ]:
# Se prueba la función 
recomendacion('"Glow Ball" - The billiard puzzle game')

____________________

### <center> Creación de un dataframe 
### <center>para ser consumido en la API con menor capacidad</center>

In [ ]:
# Se crea una nueva columna 'recomendaciones' en el DataFrame df_agrupado
# Se aplica la función recomendación a la columna 'title' para obtener recomendaciones y almacenarlas en la nueva columna
df_agrupado['recomendaciones'] = df_agrupado['title'].apply(recomendacion)

In [ ]:
# Se visualiza la tabla generada
df_agrupado

In [ ]:
# Se elimina columna que no es necesaria
df_modelado = df_agrupado[['title', 'recomendaciones']]

# Se vuelve a visualizar el dataframe
df_modelado

In [ ]:
# Se trae del df_games el ' item_id y el 'title'
df_agregar= df_games[['item_id', 'title']]

# Se visualiza el df creado
df_agregar

In [ ]:
#Se consulta los duplicados
duplicates_modelado = df_agregar[df_agregar['title'].duplicated()]

# Se visualizan los duplicados
duplicates_modelado

In [ ]:
# Se elimina duplicados en df_agregar basados en la columna 'title'
df_agregar = df_agregar.drop_duplicates(subset='title', keep='first')

#Se visualiza el df
df_agregar

In [ ]:
# Se realiza la unión de las tablas
df_merged_1 = pd.merge(df_modelado, df_agregar[['item_id', 'title' ]], on='title', how='left')

In [ ]:
# Se  selecciona solo las columnas relevantes
df_modelado = df_merged_1[['title', 'recomendaciones', 'item_id']]

# Se cuentan los valores que tiene
df_modelado.count()

In [ ]:
# SE verifican los datos
funciones.verificar_tipo_datos(df_modelado)

_______________


### <center> Se guarda en un archivo PARQUET</center>
 ### <center>para ser consumido por la API </center>

In [ ]:
#Se construye la ruta completa al archivo DATA desde la ubicación del notebook
modelo = os.path.join('..', 'DATA', 'modelo_rec.parquet')

# Guarda el DataFrame en el archivo PARQUET
df_modelado.to_parquet(modelo)

print(f'Se guardó el archivo {modelo}')

_____________________
______________

### <center> Se verifica el funcionamiento
### <center> del consumo del PARQUET 
### <center>y de la función a deployar

In [ ]:
# Se lee el archivo PARQUET
modelo = os.path.join('..', 'DATA', 'modelo_rec.parquet')
df=pd.read_parquet(modelo)

#Se  ve el contenido en un dataframe
df

### Función a deployar

In [ ]:
from fastapi.responses import JSONResponse


def recomendacion_juego(id_producto: int):
    recomendaciones = df[df['item_id'] == id_producto]['recomendaciones'].iloc[0]
    
    # Verificar si la lista de recomendaciones no está vacía
    if len(recomendaciones) > 0:
        recomendaciones_dict = {i + 1: juego for i, juego in enumerate(recomendaciones)}
        return recomendaciones_dict
    else:
        # Si no se encontraron recomendaciones para el ID, devolver un mensaje de error
        error_data = {'error': 'No se encontraron recomendaciones para el ID proporcionado'}
        return JSONResponse(content=error_data, status_code=404)

### Respuesta obtenida

In [ ]:
recomendacion_juego(449940)

### <center> FUNCIONA PERFECTO!!!</center>

______________
___________


___________
____________
___________
___________

In [20]:
# Se obtiene la ruta completa al archivo CSV desde la ubicación del notebook
csv_path = os.path.join('..', 'CSV', 'reviews_limpio.csv')

# Se lee el archivo CSV
df_reviews = pd.read_csv(csv_path, encoding='utf-8')
df_reviews=df_reviews[['item_id', 'user_id']]
df_reviews.shape[0]

48498

In [97]:
# Se obtiene la ruta completa al archivo CSV desde la ubicación del notebook
csv_path = os.path.join('..', 'CSV', 'games_limpio.csv')

# Se lee el archivo CSV
df_games = pd.read_csv(csv_path, encoding='utf-8')
df_games=df_games[['item_id', 'genres']]
df_games.shape[0]

71551

In [ ]:
df_games_id= df_games[['item_id', 'title']]

In [ ]:
df_agrupado = df_agrupado.merge(df_games_id, on='title', how='left')

df_agrupado

In [ ]:
filas_duplicadas = df_agrupado[df_agrupado.duplicated()]
print(filas_duplicadas)

In [ ]:
df_sin_duplicados = df_agrupado.drop_duplicates(subset=['item_id'])
df_sin_duplicados

In [ ]:
df_combinado = pd.merge(df_sin_duplicados, df_reviews[['item_id', 'user_id']], on='item_id', how='left')
df_agrupado

_________________________

In [13]:
df_agrupado

,title,genres
0,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie"
1,"""Barely Attuned Magic Thingy"" Staff","Action, Free to Play, Indie, Massively Multipl..."
2,"""Glow Ball"" - The billiard puzzle game","Casual, Indie, Sports, Strategy"
3,"""Just Another Day"" - Seduce Me Otome CD","Casual, Simulation"
4,"""Lethargic Sentience"" Wand","Action, Free to Play, Indie, Massively Multipl..."
...,...,...
28822,（尘沙惑设定集）Lost in Secular Love - Concept Design ...,"Adventure, Casual, Indie, Simulation"
28823,４人打ちアクション麻雀 / ACTION MAHJONG,"Action, Casual, Indie, Early Access"
28824,＜/reality＞,"Adventure, Indie"
28825,＜/reality＞ Original Soundtrack,"Adventure, Indie"


____________________________________________________
USUARIO

In [16]:
# Se agrupa por título y concatena las entradas de la columna 'genres'
df_agrupado1 = df_games_d.groupby('title')['genres'].apply(lambda x: ', '.join(x)).reset_index()

# Vuelve a incorporar la columna 'item_id' al DataFrame df_agrupado
df_agrupado1 = pd.merge(df_agrupado, df_games_d[['title', 'item_id']], on='title', how='left')
df_agrupado1

,title,genres,item_id
0,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie",449940
1,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie",449940
2,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie",449940
3,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie",449940
4,"""Barely Attuned Magic Thingy"" Staff","Action, Free to Play, Indie, Massively Multipl...",308163
...,...,...,...
71544,＜/reality＞,"Adventure, Indie",562280
71545,＜/reality＞ Original Soundtrack,"Adventure, Indie",626850
71546,＜/reality＞ Original Soundtrack,"Adventure, Indie",626850
71547,🔴 Circles,"Casual, Indie",460250


In [17]:
df_agrupado1_sin_duplicados = df_agrupado1.drop_duplicates(subset=['title', 'genres', 'item_id'])
df_agrupado1_sin_duplicados

,title,genres,item_id
0,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie",449940
4,"""Barely Attuned Magic Thingy"" Staff","Action, Free to Play, Indie, Massively Multipl...",308163
9,"""Glow Ball"" - The billiard puzzle game","Casual, Indie, Sports, Strategy",388390
13,"""Just Another Day"" - Seduce Me Otome CD","Casual, Simulation",454790
15,"""Lethargic Sentience"" Wand","Action, Free to Play, Indie, Massively Multipl...",308164
...,...,...,...
71535,（尘沙惑设定集）Lost in Secular Love - Concept Design ...,"Adventure, Casual, Indie, Simulation",541220
71539,４人打ちアクション麻雀 / ACTION MAHJONG,"Action, Casual, Indie, Early Access",575810
71543,＜/reality＞,"Adventure, Indie",562280
71545,＜/reality＞ Original Soundtrack,"Adventure, Indie",626850


In [18]:
funciones.verificar_tipo_datos(df_agrupado1)

,Columna,Tipo,NO_nulos_%,Nulos_%,Nulos
0,title,[<class 'str'>],100.0,0.0,0
1,genres,[<class 'str'>],100.0,0.0,0
2,item_id,[<class 'int'>],100.0,0.0,0


In [21]:
user_genres_n = df_agrupado1_sin_duplicados.merge(df_reviews, on='item_id', how='left')
user_genres_n

,title,genres,item_id,user_id
0,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie",449940,NaN
1,"""Barely Attuned Magic Thingy"" Staff","Action, Free to Play, Indie, Massively Multipl...",308163,NaN
2,"""Glow Ball"" - The billiard puzzle game","Casual, Indie, Sports, Strategy",388390,NaN
3,"""Just Another Day"" - Seduce Me Otome CD","Casual, Simulation",454790,NaN
4,"""Lethargic Sentience"" Wand","Action, Free to Play, Indie, Massively Multipl...",308164,NaN
...,...,...,...,...
67498,（尘沙惑设定集）Lost in Secular Love - Concept Design ...,"Adventure, Casual, Indie, Simulation",541220,NaN
67499,４人打ちアクション麻雀 / ACTION MAHJONG,"Action, Casual, Indie, Early Access",575810,NaN
67500,＜/reality＞,"Adventure, Indie",562280,NaN
67501,＜/reality＞ Original Soundtrack,"Adventure, Indie",626850,NaN


In [22]:

# Eliminar registros con valores nulos en la columna 'user_id'
user_genres = user_genres_n.dropna(subset=["user_id"])
user_genres.shape[0]

41109

In [23]:
conteo_valores_user_id = user_genres['user_id'].value_counts()
conteo_valores_user_id

user_id
HeerFuhrurVonAuchwitz    10
NanoPi                   10
ac_elite                 10
dragonlore23             10
Findoogle                10
                         ..
uncleshenny               1
76561198131841535         1
76561198010336981         1
STAELEVEN                 1
76561198051447459         1
Name: count, Length: 20677, dtype: int64

In [34]:
df_agrupado1_sin_duplicados1 = user_genres.drop_duplicates(keep='first')
user_genres=df_agrupado1_sin_duplicados1

In [79]:
#funciones.verificar_tipo_datos(user_genres)

,Columna,Tipo,NO_nulos_%,Nulos_%,Nulos
0,title,[<class 'str'>],100.0,0.0,0
1,genres,[<class 'str'>],100.0,0.0,0
2,item_id,[<class 'int'>],100.0,0.0,0
3,user_id,[<class 'str'>],100.0,0.0,0


In [69]:
#lista_juegos = [] # lista de titles del usuario
# Supongamos que df es tu DataFrame con columnas 'user_id' y 'genres'
# y user_id_X es el usuario específico para el cual quieres obtener los juegos
#user_id_X = 'NanoPi'  # Reemplaza '123' con el user_id específico

# Filtrar el DataFrame para el usuario específico
#df_usuario = user_genres[user_genres['user_id'] == user_id_X]

# Obtener los valores de la columna 'genres' para el usuario específico
#lista_juegos = df_usuario['title'].tolist()

# Imprimir la lista de juegos
#print(lista_juegos)

['Brutal Legend', 'Dust: An Elysian Tail', 'Freedom Planet', 'Half-Life: Source', 'Just Cause 2: Multiplayer Mod', 'Left 4 Dead 2', 'Magicite', 'Ori and the Blind Forest', 'Osmos', 'Transistor']


In [39]:
#perfil_usuario = data_vextorizado_df.loc[lista_juegos,:]

In [40]:
#perfil_usuario_f = perfil_usuario.mean()
#perfil_usuario_f

access          0.0
accounting      0.0
action          0.9
adventure       0.4
amp             0.0
animation       0.0
audio           0.0
casual          0.1
design          0.0
early           0.0
editing         0.0
education       0.0
free            0.0
illustration    0.0
indie           0.5
massively       0.0
modeling        0.0
multiplayer     0.0
photo           0.0
play            0.0
production      0.0
publishing      0.0
racing          0.0
rpg             0.3
simulation      0.0
software        0.0
sports          0.0
strategy        0.1
training        0.0
utilities       0.0
video           0.0
web             0.0
dtype: float64

In [41]:
# creamos un subconjunto no visto
#no_jugadas_usuario = data_vextorizado_df.drop(lista_juegos, axis=0)

In [42]:
#calculamos la similitud entre el perfil y los juegos no visto
#perfil_usuario_similitud = cosine_similarity(perfil_usuario_f.values.reshape(1,-1), no_jugadas_usuario)

In [43]:
#almacenamos en un nuevo dataframe
#perfil_usuario_similitud_df = pd.DataFrame(perfil_usuario_similitud.T, index=no_jugadas_usuario.index, columns=['Similitud Coseno'])
 

In [44]:
'''#Mostramos el resultado ordenado
ordenado_resultado = perfil_usuario_similitud_df.sort_values(by='Similitud Coseno', ascending=False)
ordenado_resultado.head(10)'''

,Similitud Coseno
title,
Totem,0.910465
Shrouded in Sanity - Original Soundtrack,0.910465
Mystik Belle,0.910465
Imprisoned Light,0.910465
Imprisoned Light - Soundtrack,0.910465
In Exilium,0.910465
In Exilium - OST,0.910465
In Verbis Virtus,0.910465
Shrouded in Sanity,0.910465


___________


In [80]:
'''import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def obtener_recomendaciones(user_id, data_vextorizado_df, user_genres):
    # Filtrar el DataFrame para el usuario específico
    df_usuario = user_genres[user_genres['user_id'] == user_id]

    # Obtener los valores de la columna 'genres' para el usuario específico
    lista_juegos = df_usuario['title'].tolist()

    # Seleccionar el perfil del usuario
    perfil_usuario = data_vextorizado_df.loc[lista_juegos].mean()

    # Crear un subconjunto de juegos no vistos
    no_jugadas_usuario = data_vextorizado_df.drop(lista_juegos, axis=0)

    # Calcular la similitud entre el perfil y los juegos no vistos
    perfil_usuario_similitud = cosine_similarity(perfil_usuario.values.reshape(1, -1), no_jugadas_usuario)

    # Almacenar en un nuevo DataFrame
    perfil_usuario_similitud_df = pd.DataFrame(perfil_usuario_similitud.T, index=no_jugadas_usuario.index, columns=['Similitud Coseno'])

    # Mostrar el resultado ordenado
    ordenado_resultado = perfil_usuario_similitud_df.sort_values(by='Similitud Coseno', ascending=False)
    
    # Devolver las 5 mejores recomendaciones
    top_recomendaciones = ordenado_resultado.head(5)

    return top_recomendaciones


'''

In [81]:
'''# Ejemplo de uso:
user_id_X = 'NanoPi'
recomendaciones = obtener_recomendaciones(user_id_X, data_vextorizado_df, user_genres)
print(recomendaciones)
''''

                                          Similitud Coseno
title                                                     
Totem                                             0.910465
Shrouded in Sanity - Original Soundtrack          0.910465
Mystik Belle                                      0.910465
Imprisoned Light                                  0.910465
Imprisoned Light - Soundtrack                     0.910465


In [47]:
'''import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def obtener_recomendaciones(user_id, data_vextorizado_df, user_genres):
    # Filtrar el DataFrame para el usuario específico
    df_usuario = user_genres[user_genres['user_id'] == user_id]

    # Obtener los valores de la columna 'genres' para el usuario específico
    lista_juegos = df_usuario['title'].tolist()

    # Seleccionar el perfil del usuario
    perfil_usuario = data_vextorizado_df.loc[lista_juegos].mean()

    # Crear un subconjunto de juegos no vistos
    no_jugadas_usuario = data_vextorizado_df.drop(lista_juegos, axis=0)

    # Calcular la similitud entre el perfil y los juegos no vistos
    perfil_usuario_similitud = cosine_similarity(perfil_usuario.values.reshape(1, -1), no_jugadas_usuario)

    # Almacenar en un nuevo DataFrame
    perfil_usuario_similitud_df = pd.DataFrame(perfil_usuario_similitud.T, index=no_jugadas_usuario.index, columns=['Similitud Coseno'])

    # Mostrar el resultado ordenado
    ordenado_resultado = perfil_usuario_similitud_df.sort_values(by='Similitud Coseno', ascending=False)
    
    # Devolver los títulos de las 5 mejores recomendaciones
    top_recomendaciones = ordenado_resultado.head(5).index.tolist()

    return top_recomendaciones

# Ejemplo de uso:
user_id_X = 'NanoPi'
recomendaciones = obtener_recomendaciones(user_id_X, data_vextorizado_df, user_genres)
print(recomendaciones)'''


['Totem', 'Shrouded in Sanity - Original Soundtrack', 'Mystik Belle', 'Imprisoned Light', 'Imprisoned Light - Soundtrack']


In [24]:
import pandas as pd
import numpy as np

# Supongamos que ya tienes un DataFrame llamado df_agrupado y la matriz de similitud llamada similitud

# Se genera una función que te da la recomendación por título
def recomendacion_para_usuario(user_id):
    # Obtener los juegos que el usuario ha visto
    juegos_vistos = user_genres[user_genres["user_id"] == user_id]["title"]

    # Inicializar un diccionario para almacenar la puntuación acumulada de cada juego recomendado
    puntuacion_recomendaciones = {}

    # Iterar sobre los juegos vistos por el usuario
    for juego in juegos_vistos:
        indice_juego = df_agrupado[df_agrupado["title"] == juego].index[0]
        distancias = similitud[indice_juego]
        juegos_recomendados = sorted(list(enumerate(distancias)), reverse=True, key=lambda x: x[1])[1:6]

        # Actualizar la puntuación acumulada de cada juego recomendado
        for juego_recomendado in juegos_recomendados:
            indice_recomendado = juego_recomendado[0]
            similitud_score = juego_recomendado[1]
            titulo_recomendado = df_agrupado.iloc[indice_recomendado]['title']

            if titulo_recomendado not in juegos_vistos:
                if titulo_recomendado not in puntuacion_recomendaciones:
                    puntuacion_recomendaciones[titulo_recomendado] = 0
                puntuacion_recomendaciones[titulo_recomendado] += similitud_score

    # Ordenar los juegos recomendados por puntuación acumulada
    juegos_recomendados = sorted(puntuacion_recomendaciones.items(), key=lambda x: x[1], reverse=True)[:5]

    return [juego[0] for juego in juegos_recomendados]

# Ejemplo de uso
user_id_ejemplo = "NanoPi"  # Reemplaza esto con el user_id que desees
recomendaciones_usuario = recomendacion_para_usuario(user_id_ejemplo)

print(f"Recomendaciones para el usuario {user_id_ejemplo}:")
print(recomendaciones_usuario)


Recomendaciones para el usuario NanoPi:
['2 Blue Orbs', '5 Blue Orbs', 'ABRACA - Imagic Games', 'ARCADE GAME SERIES: DIG DUG', 'ARCADE GAME SERIES: GALAGA']


In [25]:
user_genres

,title,genres,item_id,user_id
25,//N.P.P.D. RUSH//- The milk of Ultraviolet,"Action, Indie",270090,jdrocks
26,//N.P.P.D. RUSH//- The milk of Ultraviolet,"Action, Indie",270090,mr_pie_nice_cheese
35,0RBITALIS,"Indie, Simulation",278440,76561198024316193
45,"10,000,000","Action, Casual, Indie, RPG",227580,hecaton_john
46,"10,000,000","Action, Casual, Indie, RPG",227580,ghost-loli
...,...,...,...,...
67392,theHunter: Primal,"Action, Adventure, Simulation",322920,sdq101
67393,theHunter: Primal,"Action, Adventure, Simulation",322920,76561198051447459
67394,theHunter: Primal,"Action, Adventure, Simulation",322920,djm3h
67395,theHunter: Primal,"Action, Adventure, Simulation",322920,danielsanger


In [27]:
df_user_id = user_genres[['user_id']]
df_user_id

,user_id
25,jdrocks
26,mr_pie_nice_cheese
35,76561198024316193
45,hecaton_john
46,ghost-loli
...,...
67392,sdq101
67393,76561198051447459
67394,djm3h
67395,danielsanger


___________________________________________________________

### <center> Tabla generada user</center>
 ### <center>para ser consumido por la API </center>

In [30]:
# Agregar una nueva columna 'recomendaciones' usando la función recomendacion_para_usuario
df_user_id['recomendaciones'] = df_user_id['user_id'].apply(recomendacion_para_usuario)


In [31]:
df_user_id

,user_id,recomendaciones
25,jdrocks,"[A.V.A. Alliance of Valiant Arms™: Free Pack, ..."
26,mr_pie_nice_cheese,"[2 Blue Orbs, 5 Blue Orbs, ABRACA - Imagic Gam..."
35,76561198024316193,"[Black Rose, Blink the Bulb, Chasing Styx, Doo..."
45,hecaton_john,"[Buff Knight Advanced, Chicken Assassin - Mast..."
46,ghost-loli,"[1943 Megami Strike, 222 Hearts, 4Team, 8BitBo..."
...,...,...
67392,sdq101,"[.EXE - OST, 1001 Spikes, 24 HOURS, A Bastard'..."
67393,76561198051447459,"[Beyond Space Remastered Edition, CAR THIEF SI..."
67394,djm3h,"[AddForce, Air Guardians, Aircraft War X, Arte..."
67395,danielsanger,"[Clown2Beat Crazy Circus, Embers of Magic, Mau..."


____________________________________________________

### <center> Se guarda en un archivo PARQUET</center>
 ### <center>para ser consumido por la API </center>

In [32]:
#Se construye la ruta completa al archivo DATA desde la ubicación del notebook
modelo_u = os.path.join('..', 'DATA', 'modelo_rec_u.parquet')

# Guarda el DataFrame en el archivo PARQUET
df_user_id.to_parquet(modelo_u)

print(f'Se guardó el archivo {modelo_u}')

Se guardó el archivo ..\DATA\modelo_rec_u.parquet


_________________________________________________________________________

### <center> Se verifica el funcionamiento
### <center> del consumo del PARQUET 
### <center>y de la función a deployar

In [33]:
# Se lee el archivo PARQUET
modelo = os.path.join('..', 'DATA', 'modelo_rec_u.parquet')
df=pd.read_parquet(modelo)

#Se  ve el contenido en un dataframe
df

,user_id,recomendaciones
25,jdrocks,"[A.V.A. Alliance of Valiant Arms™: Free Pack, ..."
26,mr_pie_nice_cheese,"[2 Blue Orbs, 5 Blue Orbs, ABRACA - Imagic Gam..."
35,76561198024316193,"[Black Rose, Blink the Bulb, Chasing Styx, Doo..."
45,hecaton_john,"[Buff Knight Advanced, Chicken Assassin - Mast..."
46,ghost-loli,"[1943 Megami Strike, 222 Hearts, 4Team, 8BitBo..."
...,...,...
67392,sdq101,"[.EXE - OST, 1001 Spikes, 24 HOURS, A Bastard'..."
67393,76561198051447459,"[Beyond Space Remastered Edition, CAR THIEF SI..."
67394,djm3h,"[AddForce, Air Guardians, Aircraft War X, Arte..."
67395,danielsanger,"[Clown2Beat Crazy Circus, Embers of Magic, Mau..."


In [34]:
from fastapi.responses import JSONResponse


def recomendacion_juego_u(id_user: str):
    recomendaciones = df[df['user_id'] == id_user]['recomendaciones'].iloc[0]
    
    # Verificar si la lista de recomendaciones no está vacía
    if len(recomendaciones) > 0:
        recomendaciones_dict = {i + 1: juego for i, juego in enumerate(recomendaciones)}
        return recomendaciones_dict
    else:
        # Si no se encontraron recomendaciones para el ID, devolver un mensaje de error
        error_data = {'error': 'No se encontraron recomendaciones para el ID proporcionado'}
        return JSONResponse(content=error_data, status_code=404)

In [35]:
recomendacion_juego_u('sdq101')

{1: '.EXE - OST',
 2: '1001 Spikes',
 3: '24 HOURS',
 4: "A Bastard's Tale",
 5: "A Bastard's Tale Demo"}